# Simple Error Model for Twinkles

This notebook will calculate a simple error model for the Twinkles data as a function of the seeing and depth.

### Requirements

You will need the DESC `Monitor` and its dependencies.

You will also need the star_cache.db from  in your `Monitor/data` directory.

Finally, you need the correct opsim database for your Twinkles run. For run 1.1 this is kraken_1042 and for run 3 and beyond this is minion_1016. Both are available [here](https://www.lsst.org/scientists/simulations/opsim/opsim-v335-benchmark-surveys).

In [ ]:
import os
import desc.monitor
import matplotlib.pyplot as plt
%matplotlib inline
%load_ext autoreload
%autoreload 2

### Get truth data for visits

We first want to get all the true fluxes that we will need from the cache_star database and store them in a pandas dataframe.

In [ ]:
star_db_name = os.path.join(os.getenv('$inst_dir'), '/data/', 'star_cache.db')
dbConn = desc.monitor.StarCacheDBObj(database='../data/star_cache.db')

In [ ]:
worker = desc.monitor.TrueStars(dbConn, '../data/kraken_1042_sqlite.db')

In [ ]:
worker.get_true_stars(for_obsHistIds=[220, 230])

In [ ]:
worker.star_df